# 行列の標準形と応用

参考
* [線形代数汎論](https://amzn.asia/d/cQQdYH2): ６章

行列の一般の「標準形」を次のように定義する。

* $\mathcal{M}$：考察の対称とする行列全体の集合。例えば複素$n$次正方行列
* $\mathcal{G}$：想定する変換全体の集合（変換群）。例えば相似変換($A'=S^{-1}AS$みたいな。[LA_vector_space.ipynb](LA_vector_space.ipynb)参照。)。

まず、$A, B\in \mathcal{M}$に対して「同値関係」を次のように導入する。

$$A\sim B \Longleftrightarrow \exist T \in \mathcal{G}, T(A) = B$$

このとき$\sim$は$\mathcal{M}$を同値類に分類しており、各同値類の代表元のことを「標準形」と呼ぶ。

色んな線形写像と、その変換（標準形）について見ていきます。（岩田先生のスライドより引用）

![normal](figs/normal_form.png)


In [ ]:
import numpy as np

# 基本行列を作ります

def make_transposition_matrix(size, i, j):
    # 行列のiとj番目の行（列）を入れ替える行列を作ります
    P = np.eye(size)
    P[i, i] = 0
    P[j, j] = 0
    P[i, j] = 1
    P[j, i] = 1
    return P

M, N = 3, 4
A = np.random.randint(1, 10, (M, N))
print("### 元の行列 ###")
print(A)

print("### １行目と２行目を入れ替え ###")
P = make_transposition_matrix(M, 1, 2)
print(P @ A)

print("### １列目と２列目を入れ替え ###")
P = make_transposition_matrix(N, 1, 2)
print(A @ P)


def make_scale_matrix(size, i, s):
    # 行列のi行（列）をs倍する行列を作ります　
    T = np.eye(size)
    T[i, i] = s
    return T

print("### ２行目を２倍 ###")
T =  make_scale_matrix(M, 2, 2)
print(T @ A)


def make_add_matrix(size, i, j, s):
    # 行列のj行（列）をs倍してi行（列）に足す行列を作ります　
    E = np.eye(size)
    E[i, j] = s
    return E

print("### ２行目を３倍して１行目に追加 ###")
E =  make_add_matrix(M, 1, 2, 3)
print(E @ A)


### 元の行列 ###
[[4 3 1 3]
 [7 2 2 9]
 [8 1 5 5]]
### １行目と２行目を入れ替え ###
[[4. 3. 1. 3.]
 [8. 1. 5. 5.]
 [7. 2. 2. 9.]]
### １列目と２列目を入れ替え ###
[[4. 1. 3. 3.]
 [7. 2. 2. 9.]
 [8. 5. 1. 5.]]
### ２行目を２倍 ###
[[ 4.  3.  1.  3.]
 [ 7.  2.  2.  9.]
 [16.  2. 10. 10.]]
### ２行目を３倍して１行目に追加 ###
[[ 4.  3.  1.  3.]
 [31.  5. 17. 24.]
 [ 8.  1.  5.  5.]]


### 階数標準形

![rank_normal](figs/rank_normal_form.png)

行列の基本演算によって、適当な$m\times n$行列は上のような形に変形できる。
このとき得られる

$$A'=SAT$$

を$A$の階数標準形と呼ぶ。

これによって行列のランクは変化していないことに注意しよう。（$S$と$T$は行列の基本演算であり、正則なので。）
これを使うと、やや面倒な関係式を分かりやすく証明することが多々ある。

例えば、

$$
\operatorname{rank}(AB) + \operatorname{rank}(BC)
\leq 
\operatorname{rank}(ABC) + \operatorname{rank}(B)
$$

を証明してみよう。（$A$は$k\times l$、$B$は$l\times m$、$C$は$m\times n$の行列）　

まず、$A$と$C$を階数標準形になおしてみる。
$A'=S_1 A T_1$, $C'=S_2 C T_2$ とすると、
上の両辺を全て「$'$」がついた記号に置き換えても、両辺のrankは変化しない。

ここで、
$$A'B'=S_1 A T_1 B'$$
について、$S_1 A T_1$が左上に$1$が並んだ行列であることを考えると、$A'B'$は$B'$の部分行列として書ける。
同様にして全ての項が$B'$の部分行列として書ける。

後はごにょごにょすると証明できるぞ（p.243参照）。

In [15]:
import numpy as np

# 基本変形（左上に１を集めます）

# M x N でrank=Rの行列
R, M, N = 3, 5, 7
np.random.seed(10)
A = np.random.randint(0, 10, (R, N))
A_left = np.repeat(A[0][None, :], M - R, axis=0)
A_left = np.random.rand(M - R).reshape(-1, 1) * A_left
A = np.vstack([A, A_left])
A_origin = A.copy()


S = np.eye(M)  # 左からかける正則行列
U = np.eye(N)  # 右からかける正則行列

assert A.shape == (M, N)
print(A)

for n in range(min(M, N)):
    # ある列について、n以上の行に非零要素があるかを調べます
    nonzero_idx = np.where(A[n:, n] > 1e-5)[0]

    if len(nonzero_idx) > 0:
        # 非零要素が対角成分にくるように入れ替えます
        idx = nonzero_idx[0] + n
        P_row = make_transposition_matrix(M, idx, n)
        P_col = make_transposition_matrix(N, n, idx)
        A = P_row @ A @ P_col
        S = P_row @ S
        U = U @ P_col
    
        # (n, n)番目の項を１にします
        T = make_scale_matrix(M, n, 1 / A[n, n])
        A = T @ A
        S = T @ S
    
        # (n, n)番目の項を使って他の行を０にします
        for m in range(n+1, M):
            E = make_add_matrix(M, m, n, -A[m, n])
            A = E @ A
            S = E @ S
    
        # (n, n)番目の項を使って他の列を０にします
        for j in range(n+1, N):
            E = make_add_matrix(N, n, j, -A[n, j])
            A = A @ E
            U = U @ E


print(np.where(np.abs(A) < 1e-6, 0, A))

# 基本変形をまとめたやつで変形してみます
A = S @ A_origin @ U

print(np.where(np.abs(A) < 1e-6, 0, A))

[[9.         4.         0.         1.         9.         0.
  1.        ]
 [8.         9.         0.         8.         6.         4.
  3.        ]
 [0.         4.         6.         8.         1.         8.
  4.        ]
 [2.62688461 1.16750427 0.         0.29187607 2.62688461 0.
  0.29187607]
 [8.2599671  3.67109649 0.         0.91777412 8.2599671  0.
  0.91777412]]
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


## 正規行列と対角化、Schur標準形

Jordan標準形に行く前に、少し特殊なケースとして、$A$が正規行列なら対角化できることを述べておこう。これをするためにSchur標準形が出てくる。

**正規行列**

正規行列とユニタリ行列とHermite行列の違いに注意しましょう

* 正規行列：$AA^H = A^HA$
* ユニタリ行列（実行列なら直交行列です）：$AA^H = A^HA = I$
* Hermite行列（実行列なら対称行列です）：$A^H = A$

**Schur標準形**

複素正方行列$A$に対して、ユニタリ行列$Q$を使って、次のような上三角行列への変換を行う標準形が存在する。　

![Shur](figs/Schur_normal_form.png)

証明は[LA_matrix_definite.ipynb](LA_matrix_definite.ipynb)でやりましたが、$A$の大きさ$n$の帰納法でやります。
ここで、特に$A$が正規行列の時、$AA^H=QRQ^HQR^HQ^H$の形に書ける。このとき$R$が対角行列でなけれなばならないことを帰納法で示せばOK。


## Jordan標準形

このように正規行列なら対角化可能です。
一方で、対角化不可能な行列に対しては、Jordan標準形を使えば似たような形式にもっていけます。

任意の複素正方行列$A$に対して、
次のような行列に変形できる正則行列$S$が存在します。

![jordan](figs/Jordan.png)

より一般には、$n\times n$行列に対しては、下図のように右下に正則行列をもつ形に変形できます。

下図の一般の場合については変形ができるとしましょう（右下に正則行列を固めるように頑張れば変形できます）
これに対して、$A$の固有値を$\lambda_1$とすると、$A- \lambda_1 I$が一般の場合に変形できます（複素正方行列には固有値が存在します）。

これに$\lambda_1 I$を足せば、Jordan標準形の最初の部分が完成します。繰り返しやればJordan標準形が得られます。

## Sylvester標準形

Sylvester標準形は「二次形式」（二次の項だけでできている多項式）について考えるときに便利です。

一般の二次形式は以下で表されます。

$$\sum_{i=1}^n \sum_{j=1}^n a_{ij} x_i x_j$$ 

例えば$x_1^2 + 3 x_1 x_2 + x_2^2$は二次形式です。これは係数行列$A=[a_{ij}]$を使うと、

$$\sum_{i=1}^n \sum_{j=1}^n a_{ij} x_i x_j=x^T A x$$ 

の形で書ける。ここでさらに、$x^T A x=(x^T A x)^T = x^T A^T x$なので、

$$\sum_{i=1}^n \sum_{j=1}^n a_{ij} x_i x_j=x^T A x=\frac{1}{2}(x^T A x + x^T A^T x) = x^T \left(\frac{A + A^T}{2}\right) x $$ 

である。これより、二次形式には係数行列の対称部分しか効かない。なので、二次形式の係数行列は対称行列を仮定しても問題ない。

一方で、対称行列を含む正規行列は対角化できることを思い出そう。
対称行列$A$は適当な変形$P$によって$PAP^T = LDL^T$とできるので、係数行列$A$は次のような形式に書き直すことができる。

![sylvester](figs/sylvester_normal_form.png)

よって、
$$\sum_{i=1}^n \sum_{j=1}^n a_{ij} x_i x_j=x^T A x$$ 
に対して、
$$x^T A x= (x^TS) A' (S^*x)$$ 

なので、$x^T S$なる変数変換を施すと、任意の二次形式は$a_1' x_1^2 + a_2' x_2^2 + \cdots$のように、「キレイな」形に書き直すことができる。